# Find faults associated with exposures

Craig Lage - 21-Jul-25

In [ ]:
import os, sys
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'

client = makeEfdClient()

In [ ]:
dayObs = 20250826
start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
mtmount = getEfdData(
    client,
    "lsst.sal.MTMount.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtmount_fault = mtmount[mtmount['summaryState'] == 3]

mount_stop_track = getEfdData(
    client,
    "lsst.sal.MTMount.command_stopTracking",
    begin=start,
    end=end
)

mtaos = getEfdData(
    client,
    "lsst.sal.MTAOS.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtaos_fault = mtaos[mtaos['summaryState'] == 3]

mtptg = getEfdData(
    client,
    "lsst.sal.MTPtg.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtptg_fault = mtptg[mtptg['summaryState'] == 3]

ptg_stop_track = getEfdData(
    client,
    "lsst.sal.MTPtg.command_stopTracking",
    begin=start,
    end=end
)

mthex = getEfdData(
    client,
    "lsst.sal.MTHexapod.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mthex_fault = mthex[mthex['summaryState'] == 3]

mtcamera = getEfdData(
    client,
    "lsst.sal.MTCamera.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtcamera_fault = mtcamera[mtcamera['summaryState'] == 3]

mtrotator = getEfdData(
    client,
    "lsst.sal.MTRotator.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtrotator_fault = mtrotator[mtrotator['summaryState'] == 3]

mtrotpos = getEfdData(
    client,
    "lsst.sal.MTRotator.logevent_inPosition",
    columns=['inPosition'],
    begin=start,
    end=end
)
mtrotpos_false = mtrotpos[mtrotpos['inPosition'] == False]

faults = [ptg_stop_track, mount_stop_track, mtrotpos_false, mtmount_fault, mtaos_fault, mtptg_fault, mthex_fault, mtcamera_fault, mtrotator_fault,   ]
names = ['mtptg sent stopTracking', 'mtmount sent stopTracking', 'mtrotator sent inPosition False', 'mtmount faulted', 'mtaos faulted', 'mtptg faulted', 'mthex faulted', 'mtcamera faulted', 'mtrotator faulted']
print(len(mtmount_fault), len(mtaos_fault), len(mtptg_fault), len(mthex_fault), len(mtcamera_fault), len(mtrotator_fault), len(mtrotpos_false), len(mount_stop_track), len(ptg_stop_track))

In [ ]:
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
            where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
print(len(exposureList))
for [id,record] in exposureList:
    if record.observation_type not in ['acq', 'science']:
        continue
    dataId = {'exposure':record.id, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    exp_begin = expRecord.timespan.begin.utc
    exp_end = expRecord.timespan.end.utc
    for n, fault in enumerate(faults):
        
        for fault_time in fault.index:
            t = Time(fault_time)
            if (t > exp_begin) and (t < exp_end):
                print(f"{names[n]} during {record.id} at {t.isot}")



In [ ]:
expId = 2025082600458
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
exp_begin = expRecord.timespan.begin.utc - TimeDelta(60.0, format='sec')
exp_end = expRecord.timespan.end.utc + TimeDelta(60.0, format='sec')
for n, fault in enumerate(faults):
    for fault_time in fault.index:
        t = Time(fault_time)
        if (t > exp_begin) and (t < exp_end):
            print(f"{names[n]} during {expId} at {t.isot}")


In [ ]:
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
            where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
print(len(exposureList))
for [id,record] in exposureList:
    if record.observation_type not in ['acq', 'science']:
        continue
    dataId = {'exposure':record.id, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    exp_begin = expRecord.timespan.begin.utc
    exp_end = expRecord.timespan.end.utc
    
    for fault_time in ptg_stop_track.index:
        t = Time(fault_time)
        if (t > exp_begin) and (t < exp_end):
            print(f"MTPtg sent a stop_tracking during {record.id} at {t.isot}")
            script = getEfdData(
                    client,
                    "lsst.sal.Script.logevent_logMessage",
                    begin=exp_begin,
                    end=exp_end
                )
            for i in range(len(script)):
                if len(script['traceback'].iloc[i]) > 1000:
                    print(script['traceback'].iloc[i].split('\n')[-1])

